In [36]:
import numpy as np
import pandas as pd
from tqdm import tqdm

In [37]:
!ls /data2/yt8m/code/yt8m

notebooks  sigmoid  softmax


In [3]:
# p = np.load('/data2/yt8m/code/yt8m/probs_rnn3_tr_0919_1943.npy', allow_pickle=True) lb0.5xx

In [52]:
p1 = np.load('/data2/yt8m/code/yt8m/softmax/rnn3_ft_1005_5492_lb746.npy', allow_pickle=True)

In [53]:
p2 = np.load('/data2/yt8m/code/yt8m/softmax/rnn4_ft_1006_5625.npy', allow_pickle=True)
p1[:2]

array([[9.8455113e-01, 2.0337227e-07, 3.5731500e-07, ..., 6.0892003e-06,
        1.9596441e-06, 4.1937537e-07],
       [9.9528128e-01, 5.2664177e-06, 1.0759398e-06, ..., 1.2880225e-06,
        1.9060346e-05, 3.0180809e-06]], dtype=float32)

In [54]:
p = np.average([p1, p2], 0, weights=[0.5, 0.5])

In [65]:
#p = np.load('/data2/yt8m/code/yt8m/softmax/rnn6_mt_1004_5838.npy', allow_pickle=True)
#p = np.load('/data2/yt8m/code/yt8m/softmax/rnn3_mt_1004_5910.npy', allow_pickle=True)
#p = np.load('/data2/yt8m/code/yt8m/softmax/rnn3_ft_1005_5492.npy', allow_pickle=True)
#p = np.load('/data2/yt8m/code/yt8m/softmax/rnn4_ft_1006_5625.npy', allow_pickle=True)
p = np.load('/data2/yt8m/code/yt8m/softmax/rnn6_ft_1006_5385.npy', allow_pickle=True)



In [66]:
type(p)

numpy.ndarray

In [67]:
p[:2]

array([[6.4755529e-01, 1.3144125e-04, 1.2640365e-04, ..., 1.7654180e-05,
        1.5098701e-04, 6.9966816e-05],
       [9.9816930e-01, 7.7113407e-07, 8.1815989e-07, ..., 3.3371165e-08,
        9.1216111e-07, 4.8337146e-08]], dtype=float32)

In [68]:
df_test = pd.read_csv('/data2/yt8m/3/frame/test_ids.csv')
df_test.head()

,vid,start_time,seg_id
0,MeVj,0,MeVj:0
1,MeVj,5,MeVj:5
2,MeVj,10,MeVj:10
3,MeVj,15,MeVj:15
4,MeVj,20,MeVj:20


In [69]:
def get_classes():
    df = pd.read_csv('/data2/yt8m/3/frame/classes_1001.csv')
    c = df['class'].values.tolist()
    stoi = { c[i]: i for i in range(len(c)) }
    return c, stoi

classes, _ = get_classes()

In [70]:
classes[:2]

['none', '3']

In [71]:
print(df_test.shape, p.shape)

(2038114, 3) (2038114, 1001)


In [72]:
res = []
for i in tqdm(range(1, 1001)):
    idx = np.argsort(p[:,i])[::-1][:150000]
    best_segs = df_test.seg_id.values[idx]
    pred_str = ' '.join(best_segs)
    res.append({
        'Class': classes[i],
        'Segments': pred_str
    })


100%|██████████| 1000/1000 [06:28<00:00,  2.56it/s]


In [73]:
df_sub = pd.DataFrame(res)
df_sub.head()

,Class,Segments
0,3,UCwo:120 lJpI:210 CbCN:130 4LI3:145 9Fmm:120 1...
1,7,mXSB:70 felC:120 mXSB:115 T1ax:125 mXSB:75 mXS...
2,8,sDmH:150 sDmH:180 wBXw:210 sDmH:185 SDNb:10 bL...
3,11,kMqw:85 rpDi:260 0jJn:220 aqe8:170 pnhe:145 0Y...
4,12,hOlg:210 FF5P:40 FXxf:95 FXxf:90 hOlg:290 hOlg...


In [74]:
df_sub.to_csv('sub_rnn6_1006_5385.csv', index=False)

In [1]:
!kaggle competitions submit -c youtube8m-2019 -f sub_rnn6_1006_5385.csv -m 'submit'

100%|██████████████████████████████████████| 1.19G/1.19G [01:42<00:00, 12.4MB/s]
Successfully submitted to The 3rd YouTube-8M Video Understanding Challenge

In [ ]:
!ls -lh